In [5]:
import pandas as pd
from glob import glob
import os
import numpy as np
def clean_b_string(x):
    if isinstance(x, bytes):
        return x.decode("utf-8")
    elif isinstance(x, str) and x.startswith("b'") and x.endswith("'"):
        return x[2:-1]
    return x
def combine_aig(row):
    m, r = row['aig_m'], row['aig_r']
    if pd.isna(m) or pd.isna(r): return np.nan
    if m == "Y" and r == "Y":
        return 'B'
    elif m == "Y" and r == "N":
        return 'M'
    elif m == "N" and r == "Y":
        return 'R'
    else:
        return 'N'

In [7]:
masterbuild10 = pd.read_sas("/users/danielmatten/Desktop/m/mb_2010_pub.sas7bdat")
# List of columns to keep
masterbuild10.columns = masterbuild10.columns.str.lower()


masterbuild10['reporting_year'] = 2010
columns_to_keep = ['lea', 'schlcode','grade','ethnicity','sex','aig_m','aig_r','eds','reporting_year','mastid'] #aig_code if needed


# Keep only those columns
masterbuild10 = masterbuild10[columns_to_keep]
for col in masterbuild10.select_dtypes(include='object').columns:
    masterbuild10[col] = masterbuild10[col].map(clean_b_string)
masterbuild10.replace("XG", np.nan, inplace=True)

masterbuild10['aig'] = masterbuild10.apply(combine_aig, axis=1)
masterbuild10.drop(columns=['aig_m','aig_r'],inplace=True)
masterbuild10['ethnic'] = masterbuild10['ethnicity']
masterbuild10.drop(columns=['ethnicity'],inplace=True)
# Replace 'Y' in 'aig' with the corresponding value from 'aig_code'
#masterbuild10.loc[masterbuild10['aig'] == 'Y', 'aig'] = masterbuild10.loc[masterbuild10['aig'] == 'Y', 'aig_code']

#masterbuild10.drop(columns=['aig_code'],inplace=True)
masterbuild10.dropna(inplace=True)
masterbuild10['ethnic'] = masterbuild10['ethnic'].astype(str).str[0]
masterbuild10.to_csv("../data/raw/masterbuild10.csv", index=False)

In [12]:
raw_data_path = "../data/raw"
csv = [
    f for f in glob(os.path.join(raw_data_path, "*.csv"))
]
master_files = [f for f in csv if os.path.basename(f).startswith("masterbuild")]
master_dfs = [pd.read_csv(f) for f in master_files]
masterbuild_master = pd.concat(master_dfs, ignore_index=True)
masterbuild_master.drop(columns=['aig_code','mpw'],inplace=True)
masterbuild_master.to_csv("../data/processed/masterbuild_master.csv", index=False)


/var/folders/dk/2mw0y5w91dggrsnrpxb04jv80000gn/T/ipykernel_75700/209106515.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  master_dfs = [pd.read_csv(f) for f in master_files]
/var/folders/dk/2mw0y5w91dggrsnrpxb04jv80000gn/T/ipykernel_75700/209106515.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  master_dfs = [pd.read_csv(f) for f in master_files]
/var/folders/dk/2mw0y5w91dggrsnrpxb04jv80000gn/T/ipykernel_75700/209106515.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  master_dfs = [pd.read_csv(f) for f in master_files]
/var/folders/dk/2mw0y5w91dggrsnrpxb04jv80000gn/T/ipykernel_75700/209106515.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  master_dfs = [pd.read_csv(f) for f in master_files]
/var/folders/dk/2mw0y5w91dggrsnrpxb04jv80000gn/T/ipykernel_7